In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Introducción

En este desafío vamos a compara la performance de modelos de árboles de clasificación con el objetivo de mostrar que modelos más complejos no siempre performan mejor que los más simples. 

Vamos a entrenar varios modelos de clasificación basados en árboles de decisión para predecir si una pelicula va a obtener un premio oscar o no. 


## Dataset

En esta clase usaremos un dataset con info de películas ("Movie_classification.csv").  
Este dataset esta conformado por los siguientes features:  

 *   **Marketing expense:**    (float64)    Gasto total en Marketing      
 *   **Production expense:**   (float64)    Gasto total de Producción
 *   **Multiplex coverage:**   (float64)    Cobertura promedio de Multiplex
 *   **Budget:**               (float64)    Presupuesto
 *   **Movie_length:**         (float64)    Duración de la película
 *   **Lead_ Actor_Rating:**   (float64)    Puntaje sobre el actor principal
 *   **Lead_Actress_rating:**  (float64)    Puntaje sobre la actriz principal
 *   **Director_rating:**      (float64)    Puntaje sobre el Director
 *   **Producer_rating:**      (float64)    Puntaje sobre el Productor
 *   **Critic_rating:**        (float64)    Puntaje que le puso la crítica
 *   **Trailer_views:**        (int64)      Cantidad de vistas del Trailer
 *   **3D_available:**         (object)     Si esta disponible en 3D (Yes/No)
 *   **Time_taken:**           (float64)    Duración de la película
 *   **Twitter_hastags:**      (float64)    Cantidad de menciones en twitter
 *   **Genre:**                (object)     Genero de la película
 *   **Avg_age_actors:**       (int64)      Edad promedio de los actores
 *   **Num_multiplex:**        (int64)      Cantidad de Multiplex
 *   **Collection:**           (int64)      Recaudación
 *   **Start_Tech_Oscar:**     (int64)      Si recibió un oscar o no.
 
 

## Imports

In [ ]:
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


## Ejercicio 1

Con la preparación de datos que hicimos en la notebook de checkpoint, creemos dos archivos csv uno para los datos de train y otro para los datos de test.

Guardemos los archivos en la carpeta Data de esta clase, con los campos separados por tab.

Después del ejercicio 5 de la práctica de checkpoint, guardamos los archivos de este modo:

```
X_train.to_csv("../Data/Movie_classification_train_X.csv", sep="\t", header=True, index = False)

X_test.to_csv("../Data/Movie_classification_test_X.csv", sep="\t", header=True, index = False)

y_train.to_csv("../Data/Movie_classification_train_y.csv", sep="\t", header=True, index = False)

y_test .to_csv("../Data/Movie_classification_test_y.csv", sep="\t", header=True, index = False)
```

## Ejercicio 2

Leamos desde la carpeta Data, en instancias de DataFrame, los datos de train y test resultados del paso anterior.

In [ ]:
X_train = pd.read_csv("../Data/Movie_classification_train_X.csv", sep="\t")
X_train.head()

In [ ]:
y_train = pd.read_csv("../Data/Movie_classification_train_y.csv", sep="\t")
y_train.head()

In [ ]:
X_test = pd.read_csv("../Data/Movie_classification_test_X.csv", sep="\t")
X_test.head()

In [ ]:
y_test = pd.read_csv("../Data/Movie_classification_test_y.csv", sep="\t")
y_test.head()

## Ejercicio 3

Usando los parametros por default de `DecisionTreeClassifier`entrenemos un modelo de clasificación que prediga si una película es ganadora de Un premio Oscar

Grafiquemos el árbol obtenido y observemos la complejidad del modelo creado.

In [ ]:
my_default_tree_gini = tree.DecisionTreeClassifier(random_state=17)
my_default_tree_gini.fit(X_train, y_train)

In [ ]:
my_default_tree_gini.feature_importances_

In [ ]:
y_pred_test = my_default_tree_gini.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred_test)

In [ ]:
accuracy_score(y_test, y_pred_test)

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(my_default_tree_gini,feature_names = X_train.columns,filled=True,rounded=True, fontsize=10)
plt.show()

## Ejercicio 4

Sabemos que el modelo por default usa Gini como criterio de pureza.

Comparemos el modelo obtenido en el punto anterior con uno que use entropía. 

Para eso, entrenemos el modelo con todos los parámetros por default, excepto el criterio de pureza, y evaluemos las métricas de accuracy y confusion matrix para ambos modelos.

Grafiquemos el modelo obtenido.

¿Podemos concluir que la performance de ambos modelos es equivalente?

Estos modelos ¿emplean en la clasificación todas las variables disponibles en el dataset? ¿Difieren las variables seleccionadas en cada uno?

In [ ]:
my_default_tree_entropy = tree.DecisionTreeClassifier(criterion = "entropy", random_state=17)
my_default_tree_entropy.fit(X_train, y_train)

In [ ]:
my_default_tree_entropy.feature_importances_

In [ ]:
y_pred_test = my_default_tree_entropy.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred_test)

In [ ]:
accuracy_score(y_test, y_pred_test)

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(my_default_tree_entropy,feature_names = X_train.columns,filled=True,rounded=True, fontsize=10)
plt.show()

**¿Podemos concluir que la performance de ambos modelos es equivalente?**

Sí, los valores de accuracy y confusion matrix obtenidos son muy similares.

**Estos modelos ¿emplean en la clasificación todas las variables disponibles en el dataset? ¿Difieren las veariables seleccionadas en cada uno?**

Mirando los valores de `feature_importances_`vemos que en cada modelo hay dos variables que no se usan como predictoras, pero son distintas en cada uno:

In [ ]:
mask_gini_unused = my_default_tree_gini.feature_importances_ == 0
X_train.columns[mask_gini_unused]

In [ ]:
mask_entropy_unused = my_default_tree_entropy.feature_importances_ == 0
X_train.columns[mask_entropy_unused]

## Ejercicio 5

Elijamos las cuatro variables de mayor importancia como variables predictoras y entrenemos otros dos modelos usando gini y entropia, definiendo como profundidad máxima 7.

Comparemos la performance obtenida con los entrenados en los ejercicios anteriores.

Grafiquemos estos dos nuevos modelos.

¿Qué podemos concluir comparando los cuatro modelos entrenados?

In [ ]:
gini_importance = pd.DataFrame({'atributo':X_train.columns, 
                                'importancia': my_default_tree_gini.feature_importances_}).sort_values('importancia', ascending = False).iloc[0:4, :]
gini_importance

In [ ]:
gini_features = gini_importance.atributo.values
gini_features

In [ ]:
my_default_tree_entropy.feature_importances_

In [ ]:
X_train.columns

In [ ]:
entropy_importance = pd.DataFrame({'atributo':X_train.columns, 
                                'importancia': my_default_tree_entropy.feature_importances_}).sort_values('importancia', ascending = False).iloc[0:4, :]
entropy_importance

In [ ]:
entropy_features = entropy_importance.atributo.values
entropy_features

In [ ]:
my_tree_gini_4 = tree.DecisionTreeClassifier(random_state=17, max_depth = 7)
my_tree_gini_4.fit(X_train.loc[:, gini_features], y_train)

In [ ]:
my_tree_entropy_4 = tree.DecisionTreeClassifier(random_state=17, max_depth = 7, criterion = "entropy")
my_tree_entropy_4.fit(X_train.loc[:, entropy_features], y_train)

In [ ]:
y_pred_test_gini_4 = my_tree_gini_4.predict(X_test.loc[:, gini_features])
y_pred_test_entropy_4 = my_tree_entropy_4.predict(X_test.loc[:, entropy_features])

In [ ]:
confusion_matrix(y_pred_test_gini_4, y_pred_test)

In [ ]:
confusion_matrix(y_pred_test_entropy_4, y_pred_test)

In [ ]:
accuracy_score(y_pred_test_gini_4, y_pred_test)

In [ ]:
accuracy_score(y_pred_test_entropy_4, y_pred_test)

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(my_tree_gini_4,feature_names = X_train.columns,filled=True,rounded=True, fontsize=10)
plt.show()

In [ ]:
plt.figure(figsize=(20,20))
tree.plot_tree(my_tree_entropy_4,feature_names = X_train.columns,filled=True,rounded=True, fontsize=10)
plt.show()

## Conclusión

Vemos que 

* Los distintos criterios de pureza generan modelos distintos.

* La performance obtenida cuando sólo variamos el criterio de pureza y todos los demás parametros son iguales, es bastante parecida.

* Simplificando en extremo los árboles (sólo 4 features y limitando la altura máxima) obtenemos modelos de performance similar a los más complejos. Incluso si comparamos con el mejor árbol obtenido con cross validation en la práctica de checkpoint.
